In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K
import numpy as np

from keras.models import load_model

In [ ]:
path = os.path.join(os.getcwd(),'/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL')
files = [i for i in os.listdir(path) if os.path.isfile(os.path.join(path,i))]
print(files[1340])
files = np.asarray(files)
print(type(files))

In [ ]:
path2 = os.path.join(os.getcwd(),'/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA')
files2 = [i for i in os.listdir(path2) if os.path.isfile(os.path.join(path2,i))]
print(files2[0])
files2 = np.asarray(files2)
print(type(files))

In [ ]:
all_files = np.concatenate([files, files2])
print(len(all_files))
print(len(files))
print(len(files2))

In [ ]:
l1 = np.zeros((len(files)))
l2 = np.ones(len(files2)) 
all_labels = np.concatenate([l1, l2])
print(len(all_labels))
print(all_labels[5])
print(len(l1))

In [ ]:
len(np.zeros((len(files))))


In [ ]:
data = {'Image_path' : all_files, 'pneumonia' : all_labels}
train_df = pd.DataFrame(data)

In [ ]:
print(train_df.tail())

In [ ]:
path2 = os.path.join(os.getcwd(),'/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA')
files2 = [os.path.join(path2,i) for i in os.listdir(path2) if os.path.isfile(os.path.join(path2,i))]
files2 = np.asarray(files2)
path = os.path.join(os.getcwd(),'/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL')
files = [os.path.join(path,i) for i in os.listdir(path) if os.path.isfile(os.path.join(path,i))]
files = np.asarray(files)
all_files = np.concatenate([files, files2])
l1 = np.zeros((len(files)))
l2 = np.zeros((len(files2))) + 1
all_labels = np.concatenate([l1, l2])
data = {'Image_path' : all_files, 'pneumonia' : all_labels}
valid_df = pd.DataFrame(data)

In [ ]:
valid_df.tail()

In [ ]:
path2 = os.path.join(os.getcwd(),'/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA')
files2 = [os.path.join(path2,i) for i in os.listdir(path2) if os.path.isfile(os.path.join(path2,i))]
files2 = np.asarray(files2)
path = os.path.join(os.getcwd(),'/kaggle/input/chest-xray-pneumonia/chest_xray/test/NORMAL')
files = [os.path.join(path,i) for i in os.listdir(path) if os.path.isfile(os.path.join(path,i))]
files = np.asarray(files)
all_files = np.concatenate([files, files2])
l1 = np.zeros((len(files)))
l2 = np.zeros((len(files2))) + 1
all_labels = np.concatenate([l1, l2])
data = {'Image_path' : all_files, 'pneumonia' : all_labels}
test_df = pd.DataFrame(data)
test_df.head()

In [ ]:
test_df.tail()

In [ ]:
labels = ['normal', 'pneumonia']

In [ ]:
image_generator = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization= True)
    
    # flow from directory with specified batch size
    # and target image size
train_generator = image_generator.flow_from_directory(
    '../input/chest-xray-pneumonia/chest_xray/train',
    target_size = (160,160),
    shuffle=True,
    seed=13,
    batch_size = 32,
    class_mode = 'binary',
    subset="training")

In [ ]:
valid_generator =  image_generator.flow_from_directory(
    '../input/chest-xray-pneumonia/chest_xray/val',
    target_size = (160,160),
    shuffle=True,
    seed=13,
    batch_size = 8,
    class_mode = 'categorical',
    subset="training")

In [ ]:
test_generator =  image_generator.flow_from_directory(
    '../input/chest-xray-pneumonia/chest_xray/test',
    target_size = (160,160),
    shuffle=True,
    seed=13,
    batch_size = 32,
    class_mode = None,
    subset="training")

In [ ]:
x, y = train_generator.__getitem__(0)
plt.imshow(x[0]);

In [ ]:
np.shape(x)

In [ ]:
def compute_class_freq(labels):
    N = np.shape(labels)[0]
    positive = (np.sum(labels, axis = 0))/N
    negative = np.sum(1 - labels, axis = 0)/N
    return positive, negative

In [ ]:
freq_pos, freq_neg = compute_class_freq(train_generator.labels)
print(freq_pos)
print(freq_neg)

In [ ]:
pos_weights = freq_neg
neg_weights = freq_pos
pos_contribution = freq_pos * pos_weights 
neg_contribution = freq_neg * neg_weights

In [ ]:
def get_weighted_loss(pos_w, neg_w, epsilon = 1e-7):
    def weighted_loss(y_pred, y_true):
        loss = 0.0
        #for i in range(len(pos_w)):
        loss += K.mean(-(pos_w*y_true*K.log(y_pred + epsilon) + neg_w*(1-y_true)*K.log(1-y_pred + epsilon))) #complete this line
        return loss
    return weighted_loss

In [ ]:
base_model = tf.keras.applications.DenseNet121(weights = '../input/densenet-keras/DenseNet-BC-121-32-no-top.h5',include_top = False, input_shape = (160, 160, 3))
#(weights='imagenet', include_top = False, input_shape = (160, 160, 3))
  #  include_top=True, weights='imagenet', input_tensor=None, input_shape=None,
  #  pooling=None, classes=1000
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
last_layer = base_model.get_layer('conv5_block15_concat')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = GlobalAveragePooling2D()(last_output)
predictions = Dense(len(labels), activation="sigmoid")(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss=get_weighted_loss(pos_weights, neg_weights))

In [ ]:
history = model.fit_generator(train_generator, 
                              validation_data=valid_generator,
                              #steps_per_epoch=100,
                              validation_steps=25, 
                              epochs = 10, verbose = 2, 
                              callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                                                factor=0.1,patience=2,
                                                                                min_lr = 1e-8),
                                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss, 'r')
plt.plot(val_loss, 'b')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.ylabel("loss")
plt.xlabel("epoch")
plt.figure()


plt.show()

In [ ]:
predicted_vals = model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)

In [ ]:
y_pred = np.argmax(predicted_vals, axis=1)
print(y_pred)

In [ ]:
print(len(test_generator.labels))

In [ ]:
#auc_rocs = util.get_roc_curve(labels, predicted_vals, test_generator)
from sklearn.metrics import roc_auc_score, roc_curve
def get_roc_curve(labels, predicted_vals, generator):
    auc_roc_vals = []
    for i in range(len(labels)):
        try:
            gt = generator.labels[:, i]
            pred = predicted_vals[:, i]
            auc_roc = roc_auc_score(gt, pred)
            auc_roc_vals.append(auc_roc)
            fpr_rf, tpr_rf, _ = roc_curve(gt, pred)
            plt.figure(1, figsize=(10, 10))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.plot(fpr_rf, tpr_rf,
                     label=labels[i] + " (" + str(round(auc_roc, 3)) + ")")
            plt.xlabel('False positive rate')
            plt.ylabel('True positive rate')
            plt.title('ROC curve')
            plt.legend(loc='best')
        except:
            print(
                f"Error in generating ROC curve for {labels[i]}. "
                f"Dataset lacks enough examples."
            )
    plt.show()
    return auc_roc_vals

In [ ]:
auc_rocs = get_roc_curve(labels, predicted_vals, test_generator)

In [ ]:
threshold = 0.5
predicted_vals[predicted_vals > threshold] = 1.0
predicted_vals[predicted_vals <= threshold] = 0.0

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['normal', 'pneumonia']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
def compute_class_sens_spec(pred, label, class_num):
    # extract sub-array for specified class
    class_pred = pred[class_num]
    class_label = label[class_num]    
    # true positives
    tp = np.sum((class_label == 1) & (class_pred == 1))
    # true negatives
    tn = np.sum((class_label == 0) & (class_pred == 0))
    #false positives
    fp = np.sum((class_label == 0) & (class_pred == 1))
    # false negatives
    fn = np.sum((class_label == 1) & (class_pred == 0))
    # compute sensitivity and specificity
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    return sensitivity, specificity

In [ ]:
threshold = 0.5
predicted_vals[predicted_vals > threshold] = 1.0
predicted_vals[predicted_vals <= threshold] = 0.0

In [ ]:
predicted_vals[:, 1]
#np.shape(predicted_vals[0])

In [ ]:
sensitivity, specificity = compute_class_sens_spec(predicted_vals[0], y, 0)

print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")

In [ ]:
predicted_class_indices=np.argmax(predicted_vals,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
len(predictions)